# 本セクションの目次
1. データマネジメントとは何者のなのか？
2. メタデータとは？
3. ビッグデータ基盤にメタデータが必要な理由
4. メタデータを保存するメタデータストアとは
5. ビッグデータ基盤におけるメタデータの提供形態
6. 3種類存在するメタデータ
7. 次のセクションへの導入

# データマネジメントとは何者なのか？
データアーキテクチャ、マスタデータ、データウェアハウスなどの概念に対しての課題点や解決への糸口となるヒントを提供してくれる知識体型のことです。

データマネジメントの基本は、データ利用のことを考えデータ利用の生産性向上を目的としたデータをどのように管理したら良いのか？
という情報をまとめたものと考えると良いでしょう。

データマネジメントの一環として本コースで紹介する「メタデータ」という概念が出てきます。

※
ご興味のある方は、外部リソースにデータマネジメント関連の本をリンクしておきます。  

# メタデータとは？
メタデータとは、ビッグデータ基盤におけるメタデータは、データの設計書を表すものでデータの属性を表すデータのことです。  
例えば、30という数字だけでは人間は、30という数字が何者なのか判断することができません。  
そこで、km/hというメタデータを付けると速度制限か実際の速度なのか？どちらかに絞り込みができるようになってきます。  
そしてlimitというメタデータを付けると人は速度制限と理解することが可能になります。  

Mysqlの世界であれば　speed_limit_khみたいなカラムの定義になっているイメージを持ってもらえればと思います。

この例は後に出てくる、ビジネスメタデータの一例です。
メタデータには3種類あり、「ビジネスメタデータ」「テクニカルメタデータ」「オペレーショナルメタデータ」の3種類が存在しています。

ビジネスメタデータとは、テーブル定義が最も有名なビジネスメタデータです。  
テクニカルメタデータとは、テーブルのデータの状態であったり、データがどこに保存されているのか？といった技術的な情報を表すメタデータです  
オペレーショナルメタデータとは、データに関する5w1hを示すメタデータで、データの生成するまでにどのようなことが起こっていたのかを表現するメタデータです。  

となります。  
それぞれのメタデータの取得や説明はセクション４、５、６で行っていきます。

# ビッグデータ基盤にメタデータが必要な理由

ビッグデータ基盤におけるメタデータの役割は、究極的には生産性の向上です。

ビジネスメタデータはいわゆるテーブル定義に近いものですが、普段の開発でもER図などテーブルやデータに関する設計書を残すことが多いと思います(更新されているにしろされていないしろ)  
ビッグデータ基盤も同じです。ただし、ビッグデータ基盤は複数のシステムのデータが一堂に会するシステムで利用する人は全社員といったことになるのがほとんどです。  
そのため、「少人数の間だけで簡単な設計書を作って共有」というだけは分析という作業は進まないのです。  

特にビッグデータを取り巻くシステムが全て統一されたルールで作成されていればいいのですが実際はそうではありません。  
ちょっとした違いやちょっとした慣習が混じり合い意味不明になることがほとんどです。  

そのためにもデータに関する情報、すなわちメタデータをしっかりと整理していかなければ誰も理解できないただのデータのゴミ置き場になってしまうということです（データの沼と呼ばれることがあります）。  
ゴミを漁ってデータ生まれや性質をデータを利用する人が逐一調べていくのは非常に非効率なのと共に不可能なのでまとめた情報を保存する場所が必要になり。  
その場所がメタデータストアであり、まとめた情報がメタデータというわけになります。  

# メタデータを保存するメタデータストアとは？
メタデータストアとは、その名の通りメタデータを保存するデータストアのこと。  
ビッグデータ基盤ではこのメタデータストアとデータストアは明確に分離されていることが通例。  
例えば、メタデータストアはMysqlでデータストアはAmazon S3といった状況。  

メタデータストアといっても特別なものではなく、古くから馴染みのあるMysqlや各種クラウドベンダーが提供しているメタデータを保存するためのマネージドサービスを利用すると良いと思います。  
メタデータストアはプラットフォームによって異なる。  

オンプレであればMysqlやMariaDBが利用される。  
AWSやGCPといったクラウドであればGlue Data CaltalogやDataCatalogが利用される。  

どれを選択しても本質部分は同じなので、  
利用しているプラットフォームに合わせて選択すると良いと思います。  

本コースはより本質の理解や個人のPCで実行可能にするためにMysqlをメタデータストアとして進めていきます。

# ビッグデータ基盤におけるメタデータの提供形態
メタデータに保存されたメタデータはどのようにユーザに提供されるのか？
ビッグデータ基盤においてはメタデータの提供は２パターンで提供されることが多いです。

- API(APIといっても超高速なレスポインスを求めるようなAPIではない)
- GUI

ユーザも、エンジニア、非エンジニアが混在しているためよりエンジニア向けのAPI(非エンジニアももちろん使える)と万人向けのGUIで提供するということが通例です。

APIの場合はRESTのAPIでGET table とすればテーブルの定義がAPIとして返却されたり、共通的な指標を提供したり、GUIであればメタデータストアに保存されているデータをユーザに見やすいように整形するといった用途です。

APIで気をつける点としては、データそのものを返却してしまおうとする行為です。    
ビッグデータ基盤で管理しているデータは数TBを超えるデータを保持しています。  
そのため、そのようなデータを返却しようとすると、そもそも返却できないことが大半でできたとしても、システムやネットワークにかなりの負荷をかけることになります。  

今回は表示や取得のところまではコース内に含みませんが、Mysqlにデータを保存しておくことによってAPIとして返却することもGUIとして提供することも可能になります。


# 3種類存在するメタデータ
前述したように、メタデータには3種類のメタデータが存在しています。

「ビジネスメタデータ」「テクニカルメタデータ」「オペレーショナルメタデータ」の３種類です。

![図1.2 pyenvで設定したPythonのバージョンになっているかチェック](images/MetaData.png)
  
細かく分けると本コースでは紹介しきれないほどたくさんのメタデータの種類があります。  
本コースでは幹となる３つのメタデータと、基本となるその中に存在するいくつかのカテゴリを紹介しています。  

ビジネスメタデータ
- テーブル定義

テクニカルメタデータ
- データプロファイリング

オペレーショナルメタデータ
- 5w1h
- アクセスログ

どれかひとつ欠けるだけでも途端に組織の生産性が落ちます。  
データの利用が注目されている中一部の「強い人」に頼った構築やアウトプットの出し方では将来に所属しているデータ活用そのものが継続できなくなってしまう場合もあるのです。  
「強くない普通の人」でも自分でデータを見つけて分析を行いアウトプットを出すビッグデータ基盤に求められるのはそんなシステム構成であり、それらを支えるのがメタデータです。

# 次セクションへの導入
次のセクションよりメタデータを解説及び、ところどころ算出していきます。
そのための準備を行います。

ここから、注意！！！！  
Mysqlのデータベース(とテーブル)とSparkのデータベース(とテーブル)2つが出てきます。  
それぞれ区別して記載してありますが、この2つは別物です。  
Sparkのデータベース(とテーブル)のテーブル定義などはMysqlに格納されていますが、実データはロケーション
「/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/parquet/」に格納され分離されています。

## データの読み込みとテーブルの作成をします※
1. jinko.csvを読み込んで表示する
2. jinko.csvの内容を保存するSparkテーブルを作成する
3. jinko.csvのデータを2で作成したSparkテーブルにInsertする

※以下の講座で行った最初の準備の簡略版です。  
「データサイエンスのための前処理入門PythonとSparkで学ぶビッグデータエンジニアリング(PySpark) 速習講座」  
https://www.udemy.com/course/python-spark-pyspark/?referralCode=E67BF8B61F65866794EB


In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init()

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()


# spark.xxxxxと記載することで処理を分散させることが可能です。

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])
df=spark.read.option("multiLine", "true").option("encoding", "SJIS") \
    .csv("/Users/saitouyuuki/Desktop/src/pyspark_datamanagement_metadata/dataset/jinko.csv", header=False, sep=',', inferSchema=False,schema=struct)

data_t=df.filter("code!='都道府県コード'").filter("gengo=='平成'")

data_t.show()


In [ ]:
# 定義はMysqlに格納されています（次のセクションで確認を行います）
spark.sql(""" 
create database if not exists data_management_crush_course;
""")

In [ ]:
# テーブルの作成をします。
# 今回はCSVに一致した「jinko_table」Sparkテーブルを作成します
# 定義はMysqlに格納されています（次のセクションで確認を行います）

spark.sql(""" 
CREATE TABLE IF NOT EXISTS data_management_crush_course.jinko_table (code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
LOCATION '/Users/saitouyuuki/Desktop/src/pyspark_datamanagement_metadata/dataset/data_management_crush_course.db/jinko_table';
""")

# locationとは実データが保存される領域です。
# locationは一般に
# DB.db/テーブル名
# となることが多いです。


In [ ]:
# もう一つセクション５のためにテーブルを作成しておきます
# 作業用のためCSV形式で作成を行います

spark.sql(""" 
CREATE TABLE IF NOT EXISTS data_management_crush_course.jinko_code (code String, kenmei String)
row format delimited
fields terminated by ','
lines terminated by '\n'
LOCATION '/Users/saitouyuuki/Desktop/src/pyspark_datamanagement_metadata/dataset/data_management_crush_course.db/jinko_code';
""")

In [ ]:
# 作成したSparkテーブルの一覧を確認することができます
# 定義はMysqlに保存されているのでMysqlから内部的には引っ張り出してきています。
spark.sql("show tables in   data_management_crush_course").show(truncate=False)

In [ ]:
# テーブルの定義を確認することができます。
spark.sql("show create table data_management_crush_course.jinko_table").show(truncate=False)

In [ ]:
# テーブルにデータを登録します
# insertIntoというメソッドがありますが使いづらいのでテンポラリテーブルを作成してそこからselect insert します

data_t.createOrReplaceTempView("jinko_table_tmp")

#「データサイエンスのための前処理入門PythonとSparkで学ぶビッグデータエンジニアリング(PySpark) 速習講座」  
#https://www.udemy.com/course/python-spark-pyspark/?referralCode=E67BF8B61F65866794EB
#より、パーティションとリパーティションをつけることを忘れずに
spark.sql(""" 
Insert overwrite table data_management_crush_course.jinko_table partition(kenmei)
Select 
/** REPARTITION(1) */
code,gengo,wareki,seireki,chu,sokei,jinko_male,jinko_female,kenmei
from jinko_table_tmp
  """)

In [ ]:
# データがしっかり入っているか確認してみましょう
spark.sql("""
select kenmei,count(*) from data_management_crush_course.jinko_table group by kenmei
""").show()


In [ ]:
# jinko_codeテーブルにデータを登録します
# insertIntoというメソッドがありますが使いづらいのでテンポラリテーブルを作成してそこからselect insert します

data_t.createOrReplaceTempView("jinko_table_tmp")

#「データサイエンスのための前処理入門PythonとSparkで学ぶビッグデータエンジニアリング(PySpark) 速習講座」  
#https://www.udemy.com/course/python-spark-pyspark/?referralCode=E67BF8B61F65866794EB
#より、パーティションとリパーティションをつけることを忘れずに
spark.sql(""" 
Insert overwrite table data_management_crush_course.jinko_code
Select 
/** REPARTITION(1) */ 
case 
when kenmei='神奈川県' then 'AA'
else code
END as code,
kenmei
from jinko_table_tmp
where kenmei!='全国'
  """)

# のちの作業のために「全国」と「神奈川のコード」だけ変更します

In [ ]:
# jinko_codeテーブルの登録結果を見てみます。
spark.sql("select * from data_management_crush_course.jinko_code").show()

In [ ]:
# 最後はSparkをクローズする
spark.stop()
spark.sparkContext.stop()